$$\textbf{PLN. Proyecto Final: PeptideBert}$$
$$\textit{Y. Sarahi García Gozález}$$

In [2]:
import numpy as np
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import numpy as np
import torch
import urllib.request
import convert_encodings
from convert_encodings import m2
from transformers import BertModel, BertConfig, logging
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import wandb
from datetime import datetime
import yaml
import os
import shutil

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [4]:
conda install pytorch torchvision torchaudio -c pytorch-nightly

Channels:
 - pytorch-nightly
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [7]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


Descargar los datos

In [42]:


class PeptideBERTDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

        self.length = len(self.input_ids)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        attention_mask = self.attention_masks[idx]
        label = self.labels[idx]

        return {
            'input_ids': torch.tensor(input_id, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.float)
        }

def load_data(config):
    print(f'{"="*30}{"DATA":^20}{"="*30}')

    with np.load(f'./data/{config["task"]}/train.npz') as train,\
         np.load(f'./data/{config["task"]}/val.npz') as val,\
         np.load(f'./data/{config["task"]}/test.npz') as test:
        train_inputs = train['inputs']
        train_labels = train['labels']
        val_inputs = val['inputs']
        val_labels = val['labels']
        test_inputs = test['inputs']
        test_labels = test['labels']

    attention_mask = np.asarray(train_inputs > 0, dtype=np.float64)
    attention_mask_val = np.asarray(val_inputs > 0, dtype=np.float64)
    attention_mask_test = np.asarray(test_inputs > 0, dtype=np.float64)

    train_dataset = PeptideBERTDataset(input_ids=train_inputs, attention_masks=attention_mask, labels=train_labels)
    val_dataset = PeptideBERTDataset(input_ids=val_inputs, attention_masks=attention_mask_val, labels=val_labels)
    test_dataset = PeptideBERTDataset(input_ids=test_inputs, attention_masks=attention_mask_test, labels=test_labels)

    train_data_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True
    )

    val_data_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=config['batch_size'],
        shuffle=False
    )

    print('Batch size: ', config['batch_size'])

    print('Train dataset samples: ', len(train_dataset))
    print('Validation dataset samples: ', len(val_dataset))
    print('Test dataset samples: ', len(test_dataset))

    print('Train dataset batches: ', len(train_data_loader))
    print('Validation dataset batches: ', len(val_data_loader))
    print('Test dataset batches: ', len(test_data_loader))

    print()

    return train_data_loader, val_data_loader, test_data_loader


In [45]:

def download_hemolysis():
    urllib.request.urlretrieve(
        'https://github.com/ur-whitelab/peptide-dashboard/raw/master/ml/data/hemo-positive.npz',
        './data/hemo-positive.npz',
    )
    urllib.request.urlretrieve(
        'https://github.com/ur-whitelab/peptide-dashboard/raw/master/ml/data/hemo-negative.npz',
        './data/hemo-negative.npz',
    )


def download_solubility():
    urllib.request.urlretrieve(
        'https://github.com/ur-whitelab/peptide-dashboard/raw/master/ml/data/soluble.npz',
        './data/sol-positive.npz',
    )
    urllib.request.urlretrieve(
        'https://github.com/ur-whitelab/peptide-dashboard/raw/master/ml/data/insoluble.npz',
        './data/sol-negative.npz',
    )


def download_nonfouling():
    urllib.request.urlretrieve(
        'https://github.com/ur-whitelab/peptide-dashboard/raw/master/ml/data/human-positive.npz',
        './data/nf-positive.npz',
    )
    urllib.request.urlretrieve(
        'https://github.com/ur-whitelab/peptide-dashboard/raw/master/ml/data/human-negative.npz',
        './data/nf-negative.npz',
    )

    neg = np.load('./data/nf-negative.npz')
    np.savez(
        './data/nf-negative.npz',
        arr_0=neg['seqs'],
        weights=neg['weights']
    )


download_hemolysis()
download_solubility()
download_nonfouling()

convert_encodings.main()

Ejemplo

In [46]:
# Cargar el archivo .npz
data = np.load('/Users/ely/Desktop/proyectoNLP/PeptideBERT/data/hemo-positive.npz')

# Listar los nombres de los arrays en el archivo .npz
print("Nombres de los arrays en el archivo .npz:", data.files)

files=data.files
df = pd.DataFrame(data[files[0]])
df



Nombres de los arrays en el archivo .npz: ['arr_0']


,0,1,2,3,4,5,6,7,8,9,...,180,181,182,183,184,185,186,187,188,189
0,13,8,12,13,8,24,16,5,8,11,...,0,0,0,0,0,0,0,0,0,0
1,13,8,12,13,8,24,16,5,8,11,...,0,0,0,0,0,0,0,0,0,0
2,13,8,12,13,8,24,16,5,8,11,...,0,0,0,0,0,0,0,0,0,0
3,13,8,12,13,8,24,16,5,8,11,...,0,0,0,0,0,0,0,0,0,0
4,7,21,6,10,12,6,7,6,11,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,23,13,24,7,13,24,5,13,12,11,...,0,0,0,0,0,0,0,0,0,0
1822,13,24,7,13,24,5,13,12,11,13,...,0,0,0,0,0,0,0,0,0,0
1823,13,5,21,13,11,19,13,11,5,12,...,0,0,0,0,0,0,0,0,0,0
1824,13,21,21,13,11,19,24,8,11,12,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Cargar el archivo .npz
data = np.load('/Users/ely/Desktop/proyectoNLP/PeptideBERT/data/hemo-negative.npz')

# Listar los nombres de los arrays en el archivo .npz
print("Nombres de los arrays en el archivo .npz:", data.files)

files=data.files
df = pd.DataFrame(data[files[0]])
df


Nombres de los arrays en el archivo .npz: ['arr_0']


,0,1,2,3,4,5,6,7,8,9,...,180,181,182,183,184,185,186,187,188,189
0,13,8,12,13,8,24,16,5,8,11,...,0,0,0,0,0,0,0,0,0,0
1,13,8,12,13,8,24,16,5,8,11,...,0,0,0,0,0,0,0,0,0,0
2,13,8,12,13,8,24,16,5,8,11,...,0,0,0,0,0,0,0,0,0,0
3,13,8,12,13,8,24,16,5,8,11,...,0,0,0,0,0,0,0,0,0,0
4,7,11,24,14,15,11,12,10,21,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7485,19,5,11,12,11,5,12,7,7,12,...,0,0,0,0,0,0,0,0,0,0
7486,19,11,7,6,11,6,10,20,5,12,...,0,0,0,0,0,0,0,0,0,0
7487,7,8,8,14,11,11,12,7,6,7,...,0,0,0,0,0,0,0,0,0,0
7488,7,5,21,14,15,8,12,17,6,6,...,0,0,0,0,0,0,0,0,0,0


Separmos val,test y train

In [47]:
def split_data(task):
    with np.load(f'/Users/ely/Desktop/proyectoNLP/PeptideBERT/data/{task}-positive.npz') as pos,\
         np.load(f'/Users/ely/Desktop/proyectoNLP/PeptideBERT/data/{task}-negative.npz') as neg:
        pos_data = pos['arr_0']
        neg_data = neg['arr_0']

    input_ids = np.vstack((
        pos_data,
        neg_data
    ))

    labels = np.hstack((
        np.ones(len(pos_data)),
        np.zeros(len(neg_data))
    ))

    train_val_inputs, test_inputs, train_val_labels, test_labels = train_test_split(
        input_ids, labels, test_size=0.1
    )

    train_inputs, val_inputs, train_labels, val_labels = train_test_split(
        train_val_inputs, train_val_labels, test_size=0.1
    )

    if not os.path.exists(f'/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/Proyecto_lenguaje/data/{task}'):
        os.mkdir(f'/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/Proyecto_lenguaje/data/{task}')

    np.savez(
        f'/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/Proyecto_lenguaje/data/{task}/train.npz',
        inputs=train_inputs,
        labels=train_labels
    )

    np.savez(
        f'/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/Proyecto_lenguaje/data/{task}/val.npz',
        inputs=val_inputs,
        labels=val_labels
    )

    np.savez(
        f'/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/Proyecto_lenguaje/data/{task}/test.npz',
        inputs=test_inputs,
        labels=test_labels
    )


In [48]:
split_data('hemo')

Aumentación de datos

In [23]:
def combine(inputs, labels, new_inputs, new_labels):
    new_inputs = np.vstack(new_inputs)
    new_labels = np.hstack(new_labels)

    inputs = np.vstack((inputs, new_inputs))
    labels = np.hstack((labels, new_labels))

    return inputs, labels


def random_replace(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        num_to_replace = round(unpadded_len * factor)
        indices = np.random.choice(unpadded_len, num_to_replace, replace=False)
        ip[indices] = np.random.choice(np.arange(5, 25), num_to_replace, replace=True)

        new_inputs.append(ip)
        new_labels.append(label)

    return new_inputs, new_labels


def random_delete(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        ip = list(ip[:unpadded_len])
        num_to_delete = round(unpadded_len * factor)
        indices = np.random.choice(unpadded_len, num_to_delete, replace=False)
        for i in reversed(sorted(indices)):
            ip.pop(i)
        ip.extend([0] * (200 - len(ip)))

        new_inputs.append(np.asarray(ip))
        new_labels.append(label)

    return new_inputs, new_labels


def random_replace_with_A(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        num_to_replace = round(unpadded_len * factor)
        indices = np.random.choice(unpadded_len, num_to_replace, replace=False)
        ip[indices] = m2['A']

        new_inputs.append(ip)
        new_labels.append(label)

    return new_inputs, new_labels


def random_swap(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        ip = list(ip[:unpadded_len])
        num_to_swap = round(unpadded_len * factor)
        indices = np.random.choice(range(1, unpadded_len, 2), num_to_swap, replace=False)
        for i in indices:
            ip[i-1], ip[i] = ip[i], ip[i-1]
        ip.extend([0] * (200 - len(ip)))

        new_inputs.append(np.asarray(ip))
        new_labels.append(label)

    return new_inputs, new_labels


def random_insertion_with_A(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        ip = list(ip[:unpadded_len])
        num_to_insert = round(unpadded_len * factor)
        indices = np.random.choice(unpadded_len, num_to_insert, replace=False)
        for i in indices:
            ip.insert(i, m2['A'])
        if len(ip) < 200:
            ip.extend([0] * (200 - len(ip)))
        elif len(ip) > 200:
            ip = ip[:200]

        new_inputs.append(np.asarray(ip))
        new_labels.append(label)

    return new_inputs, new_labels


def random_masking(sequences, mask_prob=0.15, mask_token_id=0):
    masked_sequences = np.copy(sequences)
    mask = np.random.rand(*sequences.shape) < mask_prob
    masked_sequences[mask] = mask_token_id
    return masked_sequences


def augment_data(task):
    with np.load(f'./data/{task}/train.npz') as train:
        inputs = train['inputs']
        labels = train['labels']

    # new_inputs1, new_labels1 = random_replace(inputs, labels, 0.02)
    # new_inputs2, new_labels2 = random_delete(inputs, labels, 0.02)
    # new_inputs3, new_labels3 = random_replace_with_A(inputs, labels, 0.02)
    new_inputs4, new_labels4 = random_swap(inputs, labels, 0.02)
    # new_inputs5, new_labels5 = random_insertion_with_A(inputs, labels, 0.02)
    #new_inputs6, new_labels6 = random_masking(inputs, mask_prob=0.15, mask_token_id=0)

    # inputs, labels = combine(inputs, labels, new_inputs1, new_labels1)
    # inputs, labels = combine(inputs, labels, new_inputs2, new_labels2)
    # inputs, labels = combine(inputs, labels, new_inputs3, new_labels3)
    inputs, labels = combine(inputs, labels, new_inputs4, new_labels4)
    # inputs, labels = combine(inputs, labels, new_inputs5, new_labels5)
    #inputs, labels = combine(inputs, labels, new_inputs6, new_labels6)

    np.savez(
        f'./data/{task}/train.npz',
        inputs=inputs,
        labels=labels
    )





Model

In [54]:
logging.set_verbosity_error()

# Definimos la clase PeptideBERT, que hereda de torch.nn.Module (la clase base para todas las redes neuronales en PyTorch)
class PeptideBERT(torch.nn.Module):
    def __init__(self, bert_config): #constructor de la clase con parametros para bert
        #llamamos al constructor de la clase base torch.nn.Module.
        super(PeptideBERT, self).__init__()

        #cargamos el modelo preentrenado
        self.protbert = BertModel.from_pretrained(
            'Rostlab/prot_bert_bfd', 
            config=bert_config, #pasamos la onfiguracion del modelo
            ignore_mismatched_sizes=True 
        ) 
        #clasificacion
        self.head = torch.nn.Sequential( 
            torch.nn.Linear(bert_config.hidden_size, 1), #toma la salida de protVert y la convierte en un valor
            torch.nn.Sigmoid()
        )
    #definimos el metodo Fordward que especifica como procesar los datos
    def forward(self, inputs, attention_mask):
        #pasamos las entradas a través de ProtBert
        output = self.protbert(inputs, attention_mask=attention_mask)
        #usamos la salida de ProtBert como entrada a la capa de clasificacion
        return self.head(output.pooler_output)
 


In [58]:
#Creamos y configuramos el modelo basado en los parámetros especificados en el diccionario config. (viene en el git)
def create_model(config):
    bert_config = BertConfig(
        vocab_size=config['vocab_size'],
        hidden_size=config['network']['hidden_size'],
        num_hidden_layers=config['network']['hidden_layers'],
        num_attention_heads=config['network']['attn_heads'],
        hidden_dropout_prob=config['network']['dropout']
    )
    #creamos una istancia de PeptideBERT utilizando la configuración de BERT definida
    model = PeptideBERT(bert_config).to(config['device'])
    #regresamos el modelo
    return model

In [57]:
#criterio de pérdida,optimizador y el planificador de learning rate  para el entrenamiento del modelo

def cri_opt_sch(config, model):
    #criterio de loss (BinaryCrossEntropy)
    criterion = torch.nn.BCELoss()
    #optimizador AmadW
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['optim']['lr'])
    #Scheduler
    if config['sch']['name'] == 'onecycle':
        ## Durante el entrenamiento, el learning-rate empieza en un valor inicial, aumenta hasta el valor máximo especificado (max_lr), y luego disminuye nuevamente hacia el final del entrenamiento.
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=config['optim']['lr'],
            epochs=config['epochs'],
            steps_per_epoch=config['sch']['steps']
        ) #Ajusta el learning-rate utilizando un ciclo de una sola pasada
    elif config['sch']['name'] == 'lronplateau':
        ## ajusta el learning-rate basándose en el rendimiento del modelo. Específicamente, reduce la tasa de aprendizaje cuando una métrica de rendimiento ha dejado de mejorar.
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='max',
            factor=config['sch']['factor'],
            patience=config['sch']['patience']
        )# Reduce lr cuando la métrica especificada ha dejado de mejorar.

    return criterion, optimizer, scheduler


In [59]:
###Función que se encarga del proceso de entrenamiento
def train(model, dataloader, optimizer, criterion, scheduler, device):
    model.train()  # Pone el modelo en modo de entrenamiento
    total_loss = 0.0

    for batch in tqdm(dataloader):  # Itera sobre los lotes de datos en el dataloader
        inputs = batch['input_ids'].to(device)  # Mueve las entradas al dispositivo (CPU o GPU)
        attention_mask = batch['attention_mask'].to(device)  # Mueve la máscara de atención al dispositivo
        labels = batch['labels'].to(device)  # Mueve las etiquetas al dispositivo

        optimizer.zero_grad()  # Resetea los gradientes del optimizador

        logits = model(inputs, attention_mask).squeeze(1)  # Pasa las entradas a través del modelo y ajusta las dimensiones
        loss = criterion(logits, labels)  # Calcula la pérdida

        loss.backward()  # Calcula los gradientes
        optimizer.step()  # Actualiza los parámetros del modelo
        # scheduler.step()  # Si el scheduler es OneCycleLR, ajusta la tasa de aprendizaje en cada paso

        total_loss += loss.item()  # Acumula la pérdida total

    return total_loss / len(dataloader)  # Retorna la pérdida promedio por lote

In [60]:
def validate(model, dataloader, criterion, device):
    model.eval()  # Pone el modelo en modo de evaluación
    total_loss = 0.0

    ground_truth = []
    predictions = []

    for batch in tqdm(dataloader):  # Itera sobre los lotes de datos en el dataloader
        inputs = batch['input_ids'].to(device)  # Mueve las entradas al dispositivo
        attention_mask = batch['attention_mask'].to(device)  # Mueve la máscara de atención al dispositivo
        labels = batch['labels'].to(device)  # Mueve las etiquetas al dispositivo

        with torch.inference_mode():  # Desactiva el cálculo de gradientes
            logits = model(inputs, attention_mask).squeeze(1)  # Pasa las entradas a través del modelo
            loss = criterion(logits, labels)  # Calcula la pérdida

        total_loss += loss.item()  # Acumula la pérdida total
        # Genera predicciones binarias
        preds = torch.where(logits > 0.5, 1, 0)  
        predictions.extend(preds.cpu().tolist())  # Añade las predicciones a la lista
        ground_truth.extend(labels.cpu().tolist())  # Añade las etiquetas reales a la lista

    total_loss = total_loss / len(dataloader)  # Calcula la pérdida promedio
    accuracy = 100 * accuracy_score(ground_truth, predictions)  # Calcula la precisión

    return total_loss, accuracy  # Retorna la pérdida promedio y la precisión

In [61]:
def test(model, dataloader, device):
    model.eval()  # Pone el modelo en modo de evaluación

    ground_truth = []
    predictions = []

    for batch in tqdm(dataloader):  # Itera sobre los lotes de datos en el dataloader
        inputs = batch['input_ids'].to(device)  # Mueve las entradas al dispositivo
        attention_mask = batch['attention_mask'].to(device)  # Mueve la máscara de atención al dispositivo
        labels = batch['labels']  # Las etiquetas permanecen en la CPU

        with torch.inference_mode():  # Desactiva el cálculo de gradientes
            logits = model(inputs, attention_mask).squeeze(1)  # Pasa las entradas a través del modelo

        preds = torch.where(logits > 0.5, 1, 0)  # Genera predicciones binarias
        predictions.extend(preds.cpu().tolist())  # Añade las predicciones a la lista
        ground_truth.extend(labels.tolist())  # Añade las etiquetas reales a la lista

    accuracy = 100 * accuracy_score(ground_truth, predictions)  # Calcula la precisión

    return accuracy  # Retorna la precisión


In [ ]:
def train_model():
    print(f'{"="*30}{"TRAINING":^20}{"="*30}')

    best_acc = 0 #inicializa la mejor precisión en cero

    #iteramos cada época
    for epoch in range(config['epochs']):

        #llamamos a la funcion de entrenamiento
        train_loss = train(model, train_data_loader, optimizer, criterion, scheduler, device)
        #obtenemos learning rate
        curr_lr = optimizer.param_groups[0]['lr']
        #imprimimos loss de entrenamiento y learning rate
        print(f'Epoch {epoch+1}/{config["epochs"]} - Train Loss: {train_loss}\tLR: {curr_lr}')
        #imprimimos loss y accuracy de validacion
        val_loss, val_acc = validate(model, val_data_loader, criterion, device)
        print(f'Epoch {epoch+1}/{config["epochs"]} - Validation Loss: {val_loss}\tValidation Accuracy: {val_acc}\n')
        #Actualizar el Scheduler:
        scheduler.step(val_acc)

        #Registrar Métricas con wandb
        if not config['debug']:
            wandb.log({
                'train_loss': train_loss, 
                'val_loss': val_loss, 
                'val_accuracy': val_acc, 
                'lr': curr_lr
            })
        #Guardamos mejor modelo
        if val_acc >= best_acc and not config['debug']:
            best_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'acc': val_acc, 
                'lr': curr_lr
            }, f'{save_dir}/model.pt')
            print('Model Saved\n')
    wandb.finish()


##Configuración y Preparación###
#llamamos al archivo donde se guarda la config del modelo peptidebert
config = yaml.load(open('./config.yaml', 'r'), Loader=yaml.FullLoader)
config['device'] = device

#xargamos los datos
train_data_loader, val_data_loader, test_data_loader = load_data(config)
config['sch']['steps'] = len(train_data_loader)
#creamos el modelo
model = create_model(config)

#configuramos criterio de pérdida, optimizador y scheduler
criterion, optimizer, scheduler = cri_opt_sch(config, model)


#Configuración de Weights & Biases (WandB)
if not config['debug']:
    run_name = f'{config["task"]}-{datetime.now().strftime("%m%d_%H%M")}'
    wandb.init(project='PeptideBERT', name=run_name)

    save_dir = f'./checkpoints/{run_name}'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    shutil.copy('./config.yaml', f'{save_dir}/config.yaml')
    shutil.copy('./model/network.py', f'{save_dir}/network.py')

#Entrenamiento del Modelo
train_model()
if not config['debug']:
    model.load_state_dict(torch.load(f'{save_dir}/model.pt')['model_state_dict'], strict=False)

#test
test_acc = test(model, test_data_loader, device)
print(f'Test Accuracy: {test_acc}%')